# フレア検出コード 学習ノートブック

このノートブックでは、`BaseFlareDetector` と恒星ごとの `FlareDetector_*` クラス（例: `FlareDetector_EK_Dra`）を使って、
TESS のフレア検出・可視化の基本的な流れを学びます。

- このプロジェクトのコード構造と役割
- EK Dra の TESS FITS データの読み込み
- フレア検出器クラスによる処理フロー
- 代表的な可視化（光度曲線・エネルギー分布）
- 複数ファイル（複数セクタ）の一括処理の入り口

を順に体験できることを目標にしています。


## このノートブックの前提

- このリポジトリ直下（`kyoto-flare-detection/`）で仮想環境を有効化し、
  `jupyter lab` や `jupyter notebook` から `notebooks/flare_learning_overview.ipynb` を開くことを想定しています。
- `data/TESS/EK_Dra/` 以下に TESS の FITS ファイルが配置されている前提です。
- フレア検出のコアロジックは `src/base_flare_detector.py` と
  各恒星向けラッパークラス（例: `src/flarepy_EK_Dra.py`）に実装されています。

以下では、最初に EK Dra を題材にして一連の流れを体験し、
その後 DS Tuc A / V889 Her など他の恒星へ応用できる形にします。


## 1. セットアップ

最初に、このノートブックからプロジェクトルートを参照できるようにし、
必要なライブラリをインポートします。


In [1]:
import os
import sys
import re
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

NOTEBOOK_DIR = Path().resolve()
PROJECT_ROOT = NOTEBOOK_DIR.parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print("PROJECT_ROOT:", PROJECT_ROOT)

PROJECT_ROOT: /Users/daisukeyamashiki/Documents/kyoto-flare-detection


## 2. EK Dra の FITS ファイル一覧

このプロジェクトでは、`data/TESS/EK_Dra/` に TESS の light curve FITS ファイルが
配置されている想定です。まずはファイルの数とファイル名をざっくり確認します。


In [2]:
DATA_DIR = PROJECT_ROOT / "data" / "TESS" / "EK_Dra"
if not DATA_DIR.exists():
    print("Warning: データディレクトリが見つかりません:", DATA_DIR)

fits_files = sorted(DATA_DIR.glob("*.fits"))
print("found", len(fits_files), "FITS files")
fits_files[:5]

found 12 FITS files


[PosixPath('/Users/daisukeyamashiki/Documents/kyoto-flare-detection/data/TESS/EK_Dra/tess2019198215352-s0014-0000000159613900-0150-s_lc.fits'),
 PosixPath('/Users/daisukeyamashiki/Documents/kyoto-flare-detection/data/TESS/EK_Dra/tess2019226182529-s0015-0000000159613900-0151-s_lc.fits'),
 PosixPath('/Users/daisukeyamashiki/Documents/kyoto-flare-detection/data/TESS/EK_Dra/tess2019253231442-s0016-0000000159613900-0152-s_lc.fits'),
 PosixPath('/Users/daisukeyamashiki/Documents/kyoto-flare-detection/data/TESS/EK_Dra/tess2020020091053-s0021-0000000159613900-0167-s_lc.fits'),
 PosixPath('/Users/daisukeyamashiki/Documents/kyoto-flare-detection/data/TESS/EK_Dra/tess2020049080258-s0022-0000000159613900-0174-s_lc.fits')]

## 3. FlareDetector_EK_Dra による単一ファイルの処理

次に、1 つの FITS ファイルを選び、`FlareDetector_EK_Dra` を用いて
フレア検出とエネルギー計算を一括で実行します。


In [3]:
from src.flarepy_EK_Dra import FlareDetector_EK_Dra

example_file = fits_files[0] if fits_files else None
print("example_file:", example_file)

detector = None
if example_file is not None:
    detector = FlareDetector_EK_Dra(
        file=str(example_file),
        process_data=True,
        ene_thres_low=5e33,
        ene_thres_high=2e40,
    )

detector

example_file: /Users/daisukeyamashiki/Documents/kyoto-flare-detection/data/TESS/EK_Dra/tess2019198215352-s0014-0000000159613900-0150-s_lc.fits


## 4. 検出結果のサマリを確認する

インスタンスに計算結果が格納されているので、
フレア数や総フレアエネルギー、観測時間などの要約を確認します。


In [4]:
if detector is None:
    print("detector がまだ作成されていません。上のセルを先に実行してください。")
else:
    # 簡単なテキストサマリ
    detector.show_variables()

    # DataFrame でも確認しやすいよう、辞書にまとめる
    summary = {
        "file": detector.file,
        "flare_number": detector.flare_number,
        "sum_flare_energy": detector.sum_flare_energy,
        "precise_obs_time": detector.precise_obs_time,
        "flare_ratio": getattr(detector, "flare_ratio", None),
        "rotation_period": detector.per,
    }
    pd.DataFrame([summary])

-------- Instance Summary --------
file: /Users/daisukeyamashiki/Documents/kyoto-flare-detection/data/TESS/EK_Dra/tess2019198215352-s0014-0000000159613900-0150-s_lc.fits
tessBJD_length: 18522
flare_number: 5
sum_flare_energy: 8.297544068872356e+34
precise_obs_time: 25.89394807392273
flare_ratio: 0.0

-------- Class Summary --------
array_flare_ratio length: 1
average_flare_ratio: 0.19309531268564636


## 5. 光度曲線の可視化（Plotly Express）

`detector` にはノーマライズ済み光度配列 `mPDCSAPflux` と時刻配列 `tessBJD` が
格納されています。まずはそれらをそのまま Plotly Express で可視化してみます。


In [5]:
if detector is None or detector.tessBJD is None:
    print("detector がまだ作成されていないか、tessBJD が空です。")
else:
    fig = px.line(
        x=detector.tessBJD,
        y=detector.mPDCSAPflux,
        labels={"x": "BJD - 2457000", "y": "Normalized Flux"},
        title="EK Dra Light Curve (Plotly Express)",
    )
    fig.show()

## 6. フレアエネルギーの可視化

`BaseFlareDetector` はフレア検出後にフレア毎のエネルギー配列 `energy` と、
ピーク時刻 `peaktime` を保持しています。ここでは：

- エネルギー vs 時刻（散布図）
- エネルギー分布のヒストグラム

を簡易にプロットしてみます。


In [6]:
if detector is None or detector.energy is None or len(detector.energy) == 0:
    print("エネルギー配列がまだ計算されていないか、フレアが検出されていません。")
else:
    # 時刻 vs エネルギー
    fig_scatter = px.scatter(
        x=detector.peaktime,
        y=detector.energy,
        labels={"x": "BJD - 2457000", "y": "Flare Energy [erg]"},
        title="Flare Energy vs Time (EK Dra)",
    )
    fig_scatter.update_yaxes(type="log")
    fig_scatter.show()

    # エネルギーヒストグラム
    fig_hist = px.histogram(
        x=detector.energy,
        nbins=40,
        labels={"x": "Flare Energy [erg]", "y": "Count"},
        title="Flare Energy Distribution (EK Dra)",
    )
    fig_hist.update_xaxes(type="log")
    fig_hist.show()

## 7. 複数 FITS ファイルをまとめて処理する

次に、EK Dra の複数セクタをまとめて処理し、
各ファイルについてフレア数や総フレアエネルギーをテーブルとして確認してみます。

※ 計算コストがそれなりにかかる場合があるので、ここでは先頭数ファイルに限定します。


In [7]:
max_files = 5  # 必要に応じて変更してください

multi_detectors = []
rows = []

for file_path in fits_files[:max_files]:
    det = FlareDetector_EK_Dra(
        file=str(file_path),
        process_data=True,
        ene_thres_low=5e33,
        ene_thres_high=2e40,
    )
    multi_detectors.append(det)
    rows.append(
        {
            "file": Path(det.file).name,
            "flare_number": det.flare_number,
            "sum_flare_energy": det.sum_flare_energy,
            "precise_obs_time": det.precise_obs_time,
            "flare_ratio": getattr(det, "flare_ratio", None),
        }
    )

summary_df = pd.DataFrame(rows)
summary_df

,file,flare_number,sum_flare_energy,precise_obs_time,flare_ratio
0,tess2019198215352-s0014-0000000159613900-0150-...,5,8.297544e+34,25.893948,0.0
1,tess2019226182529-s0015-0000000159613900-0151-...,5,1.038307e+35,24.959612,0.0
2,tess2019253231442-s0016-0000000159613900-0152-...,2,2.027268e+34,23.366922,0.0
3,tess2020020091053-s0021-0000000159613900-0167-...,10,1.564255e+35,26.329596,0.0
4,tess2020049080258-s0022-0000000159613900-0174-...,5,6.672859e+34,23.443043,0.0


## 8. 他の恒星（DS Tuc A / V889 Her）への応用

ここまでの流れは、基本的に「恒星ごとのクラスとデータパスを変えるだけ」で
DS Tuc A や V889 Her にも適用できます。

- DS Tuc A: `src.flarepy_DS_Tuc_A.FlareDetector_DS_Tuc_A`
  - 連星系補正（主星 + 伴星）
  - トランジット除去を `remove()` で実装
  - `gap_threshold = 0.05` を使用
- V889 Her: `src.flarepy_V889_Her.FlareDetector_V889_Her`
  - 高度なデトレンド（`detrend_flux()` のオーバーライド）
  - `f_cut_lowpass=30`, `f_cut_spline=40` など、カットオフ周波数が異なる
  - `gap_threshold = 0.004` を使用

EK Dra で使ったパターンをそのまま流用して、

1. `DATA_DIR` を `data/TESS/DS_Tuc_A` あるいは `data/TESS/V889_Her` に変更
2. インポートとクラス名を対応する `FlareDetector_*` に変更
3. 可視化セルをそのまま流用

とすることで、他の恒星のフレア分布も同じ形で比較できます。


## 9. 次のステップ例

このノートブックは、フレア検出コードの「入口」として最低限の流れを示すことを目的としています。
より深く理解するためには、例えば次のような拡張が考えられます。

- `src/base_flare_detector.py` の `process_data()` の中身を追いながら、
  各ステップ（ギャップ補正・デトレンド・誤差再推定・検出ロジック）を可視化する。
- `run_process_data_2=True` オプションを使って、トランジット除去をスキップした
  フローとの違いを比較する（DS Tuc A など）。
- `plot_flare_matplotlib()` / `plot_energy_matplotlib()` を使って、
  論文用の図の生成フローを確認する。
- `BaseFlareDetector` が保持しているクラス変数
  (`array_flare_ratio`, `array_energy_ratio` など) を用いて、
  複数セクタ・複数恒星をまとめた統計解析ノートを作る。


## Appendix A. `process_data` の内部ステップを覗いてみる（EK Dra）

ここでは、EK Dra を題材に `process_data()` の内部で行われている処理を、
いくつかの代表的なステップに分けて可視化してみます。

- Step 0: FITS 読み込み直後の正規化光度（`load_TESS_data()` 完了時点）
- Step 1: ギャップ補正後の光度（`apply_gap_correction()`）
- Step 2: デトレンド後の光度と検出されたフレアピーク

※ 本セクションは「仕組みの理解」が目的なので、
すでに上で EK Dra のデータ一覧を取得できていることを前提にしています。


In [8]:
# Appendix A: Step 0 - FITS 読み込み直後の正規化光度
from src.flarepy_EK_Dra import FlareDetector_EK_Dra

if "fits_files" not in globals() or not fits_files:
    print("先にセクション 2 で EK Dra の FITS ファイル一覧を取得してください。")
else:
    example_file_for_debug = fits_files[0]
    print("example_file_for_debug:", example_file_for_debug)

    # process_data=False として、内部ステップを自前で呼び出すためのインスタンスを用意
    debug_detector = FlareDetector_EK_Dra(
        file=str(example_file_for_debug),
        process_data=False,
        ene_thres_low=5e33,
        ene_thres_high=2e40,
    )

    fig_raw = px.line(
        x=debug_detector.atessBJD,
        y=debug_detector.amPDCSAPflux,
        labels={"x": "BJD - 2457000", "y": "Normalized Flux"},
        title="Appendix A - Step 0: Raw normalized flux (after load_TESS_data)",
    )
    fig_raw.show()

example_file_for_debug: /Users/daisukeyamashiki/Documents/kyoto-flare-detection/data/TESS/EK_Dra/tess2019198215352-s0014-0000000159613900-0150-s_lc.fits


In [9]:
# Appendix A: Step 1 - ギャップ補正の効果を見る
if "debug_detector" not in globals():
    print("先に Appendix A の Step 0 セルを実行して debug_detector を作成してください。")
else:
    debug_detector.apply_gap_correction()

    fig_gap = go.Figure()
    fig_gap.add_trace(
        go.Scatter(
            x=debug_detector.atessBJD,
            y=debug_detector.amPDCSAPflux,
            mode="lines",
            name="original (atessBJD / amPDCSAPflux)",
            line=dict(color="gray", width=1),
        )
    )
    fig_gap.add_trace(
        go.Scatter(
            x=debug_detector.gtessBJD,
            y=debug_detector.gmPDCSAPflux,
            mode="lines",
            name="gap-corrected (gtessBJD / gmPDCSAPflux)",
            line=dict(color="blue", width=1),
        )
    )
    fig_gap.update_layout(
        title="Appendix A - Step 1: Before/after gap correction (EK Dra)",
        xaxis_title="BJD - 2457000",
        yaxis_title="Normalized Flux",
    )
    fig_gap.show()

In [10]:
# Appendix A: Step 2 - デトレンド後の光度とフレアピーク
if "debug_detector" not in globals():
    print("先に Appendix A の Step 0 セルを実行して debug_detector を作成してください。")
else:
    # デトレンド・誤差再推定・フレア検出一式を手動で実行
    debug_detector.detrend_flux()
    debug_detector.reestimate_errors()
    debug_detector.flaredetect()
    debug_detector.flaredetect_check()
    debug_detector.calculate_precise_obs_time()
    debug_detector.flare_energy(
        energy_threshold_low=5e33,
        energy_threshold_high=2e40,
    )
    debug_detector.flux_diff()
    debug_detector.rotation_period()

    # デトレンド後の光度にフレアピークを重ねて表示
    fig_det = px.line(
        x=debug_detector.tessBJD,
        y=debug_detector.s2mPDCSAPflux,
        labels={"x": "BJD - 2457000", "y": "Detrended Flux"},
        title="Appendix A - Step 2: Detrended flux and flare peaks (EK Dra)",
    )
    if debug_detector.peaktime is not None:
        for peak in debug_detector.peaktime:
            fig_det.add_vline(x=peak, line_color="red", line_dash="dash")
    fig_det.show()

    # 検出結果の要約
    flare_ratio_val = (
        debug_detector.flare_number / debug_detector.precise_obs_time
        if debug_detector.precise_obs_time > 0
        else None
    )
    print("検出フレア数:", debug_detector.flare_number)
    print("総フレアエネルギー [erg]:", debug_detector.sum_flare_energy)
    print("精密観測時間 [day]:", debug_detector.precise_obs_time)
    print("フレア発生率 [1/day]:", flare_ratio_val)
    print("回転周期 [day]:", debug_detector.per)

検出フレア数: 5
総フレアエネルギー [erg]: 8.297544068872356e+34
精密観測時間 [day]: 25.89394807392273
フレア発生率 [1/day]: 0.19309531268564636
回転周期 [day]: 2.648040254237288


## Appendix B. クラス変数を使ったセクタ間統計の例

`BaseFlareDetector` では、各ファイル（セクタ）ごとの結果をクラス変数として
蓄積する仕組みがあります。

- `array_flare_ratio`: フレア発生率（フレア数 / 観測時間）
- `array_energy_ratio`: 単位時間あたりの総フレアエネルギー
- `array_observation_time`: 観測の代表時刻（BJD の中央値）

ここでは、セクション 7 で作成した `multi_detectors` などにより
すでに蓄積されたクラス変数を簡単に可視化してみます。


In [11]:
from src.base_flare_detector import BaseFlareDetector

n_entries = len(BaseFlareDetector.array_flare_ratio)
print("BaseFlareDetector.array_flare_ratio length:", n_entries)

if n_entries == 0:
    print("まだ process_data を実行した detector がありません。セクション 3 や 7 を先に実行してください。")
else:
    df_class = pd.DataFrame(
        {
            "median_bjd": BaseFlareDetector.array_observation_time,
            "flare_ratio": BaseFlareDetector.array_flare_ratio,
            "energy_ratio": BaseFlareDetector.array_energy_ratio,
        }
    )
    df_class["median_bjd_rel"] = df_class["median_bjd"] - 2457000
    display(df_class)

    fig_rate = px.scatter(
        df_class,
        x="median_bjd_rel",
        y="flare_ratio",
        labels={"median_bjd_rel": "Median BJD - 2457000", "flare_ratio": "Flare Rate [1/day]"},
        title="Appendix B: Flare rate per file (class variables)",
    )
    fig_rate.show()

    fig_energy = px.scatter(
        df_class,
        x="median_bjd_rel",
        y="energy_ratio",
        labels={"median_bjd_rel": "Median BJD - 2457000", "energy_ratio": "Sum Flare Energy / day"},
        title="Appendix B: Sum flare energy per file (class variables)",
    )
    fig_energy.show()

BaseFlareDetector.array_flare_ratio length: 6


,median_bjd,flare_ratio,energy_ratio,median_bjd_rel
0,1696.321001,0.193095,3.204434e+33,-2.455304e+06
1,1696.321001,0.193095,3.204434e+33,-2.455304e+06
2,1723.828301,0.200324,4.159948e+33,-2.455276e+06
3,1750.334627,0.085591,8.675804e+32,-2.455250e+06
4,1883.645866,0.379801,5.941050e+33,-2.455116e+06
5,1912.055825,0.213283,2.846413e+33,-2.455088e+06


## Appendix C. DS Tuc A / V889 Her のミニマル実行例

最後に、DS Tuc A / V889 Her について、
EK Dra で行った処理フローの「最小例」を示します。

- DS Tuc A: トランジット除去や連星補正を含むクラス
- V889 Her: 高度なデトレンドを実装したクラス

ここでは、各恒星について「1 ファイルだけ処理してサマリを確認する」
ところまでにとどめ、詳細な解析や可視化は読者の演習とします。


In [12]:
from src.flarepy_DS_Tuc_A import FlareDetector_DS_Tuc_A
from src.flarepy_V889_Her import FlareDetector_V889_Her

DATA_DS_TUC = PROJECT_ROOT / "data" / "TESS" / "DS_Tuc_A"
DATA_V889 = PROJECT_ROOT / "data" / "TESS" / "V889_Her"

print("DS_Tuc_A data dir:", DATA_DS_TUC, "exists:", DATA_DS_TUC.exists())
print("V889_Her data dir:", DATA_V889, "exists:", DATA_V889.exists())

DS_Tuc_A data dir: /Users/daisukeyamashiki/Documents/kyoto-flare-detection/data/TESS/DS_Tuc_A exists: True
V889_Her data dir: /Users/daisukeyamashiki/Documents/kyoto-flare-detection/data/TESS/V889_Her exists: False


In [13]:
# Appendix C: DS Tuc A の最小例
if not DATA_DS_TUC.exists():
    print("DS Tuc A のデータディレクトリが見つかりません:", DATA_DS_TUC)
else:
    ds_files = sorted(DATA_DS_TUC.glob("*.fits"))
    print("DS Tuc A FITS files:", len(ds_files))

    if not ds_files:
        print("FITS ファイルが存在しないため、このセルはスキップします。")
    else:
        ds_example = ds_files[0]
        print("ds_example:", ds_example)

        ds_detector = FlareDetector_DS_Tuc_A(
            file=str(ds_example),
            process_data=True,
            ene_thres_low=5e33,
            ene_thres_high=2e40,
        )
        ds_detector.show_variables()

DS Tuc A FITS files: 5
ds_example: /Users/daisukeyamashiki/Documents/kyoto-flare-detection/data/TESS/DS_Tuc_A/tess2018206045859-s0001-0000000410214986-0120-s_lc.fits
-------- Instance Summary --------
file: /Users/daisukeyamashiki/Documents/kyoto-flare-detection/data/TESS/DS_Tuc_A/tess2018206045859-s0001-0000000410214986-0120-s_lc.fits
tessBJD_length: 18040
flare_number: 6
sum_flare_energy: 1.145365094916553e+35
precise_obs_time: 26.524874270412965
flare_ratio: 0.0

-------- Class Summary --------
array_flare_ratio length: 7
average_flare_ratio: 0.21305596165641466


In [14]:
# Appendix C: V889 Her の最小例
if not DATA_V889.exists():
    print("V889 Her のデータディレクトリが見つかりません:", DATA_V889)
else:
    v889_files = sorted(DATA_V889.glob("*.fits"))
    print("V889 Her FITS files:", len(v889_files))

    if not v889_files:
        print("FITS ファイルが存在しないため、このセルはスキップします。")
    else:
        v889_example = v889_files[0]
        print("v889_example:", v889_example)

        v889_detector = FlareDetector_V889_Her(
            file=str(v889_example),
            process_data=True,
            ene_thres_low=5e33,
            ene_thres_high=2e40,
        )
        v889_detector.show_variables()

V889 Her のデータディレクトリが見つかりません: /Users/daisukeyamashiki/Documents/kyoto-flare-detection/data/TESS/V889_Her
